In [1]:
# @title Data retrieval
import os, requests
#download data 
fname = 'motor_imagery.npz'
url = "https://osf.io/ksqv8/download"

if not os.path.isfile(fname):
  try:
    r = requests.get(url)
  except requests.ConnectionError:
    print("!!! Failed to download data !!!")
  else:
    if r.status_code != requests.codes.ok:
      print("!!! Failed to download data !!!")
    else:
      with open(fname, "wb") as fid:
        fid.write(r.content)

In [2]:
# @title Install packages (`nilearn`, `nimare`. `duecredit`), import `matplotlib` and set defaults
# install packages to visualize brains and electrode locations
!pip install nilearn --quiet
!pip install nimare --quiet
!pip install duecredit --quiet

from matplotlib import rcParams
from matplotlib import pyplot as plt
import pandas as pd 
from sklearn import datasets
from sklearn import svm
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error , confusion_matrix, accuracy_score

rcParams['figure.figsize'] = [20, 4]
rcParams['font.size'] = 15
rcParams['axes.spines.top'] = False
rcParams['axes.spines.right'] = False
rcParams['figure.autolayout'] = True

     |████████████████████████████████| 9.6 MB 4.0 MB/s 
     |████████████████████████████████| 13.3 MB 2.4 MB/s 
     |████████████████████████████████| 3.2 MB 46.8 MB/s 
     |████████████████████████████████| 86 kB 2.7 MB/s 
     |████████████████████████████████| 179 kB 28.9 MB/s 


In [3]:
# @title Data loading
import numpy as np
from pickle import TRUE
alldat = np.load(fname, allow_pickle=True)['dat']

# select just one of the recordings here. 11 is nice because it has some neurons in vis ctx.
dat1 = alldat[0][0]
dat2 = alldat[0][1]
# get trial number
# print(len(dat1['stim_id']))

#get trial number
# print(alldat.shape)

In [4]:
# function to get subsets of data
def get_sub_dat(alldata, sub_id, exp_id):
  return alldata[sub_id][exp_id]

In [5]:
def dftime_win(dat):
  data = {'t_on': dat['t_on'], 't_off': dat['t_off'], 'stim_id': dat['stim_id']}
  dftime = pd.DataFrame(data)
  return dftime

In [6]:
def dat_frame_t(dat, channel_n = 46):
  #get dft
  a = dat['V']
  epoch_id=[]
  for i in range(channel_n):
    epoch_id.append(i+1)
  epoch_id_array = np.array(epoch_id)
  df = pd.DataFrame(a, columns=epoch_id)
  dft = df.T
  return dft

In [7]:
#generate X for SVM
def slice_chan_dat(dft,dftime,channel_i):
  chan_i = dft.loc[channel_i]
  chan_array = []
  for j in range (0, len(dftime)):
    chan_array.append(chan_i[dftime["t_on"][j]: dftime["t_off"][j]])
  # print(chan_array.shape)
  # ar_chan_i = chan_array.values
  result = np.array([np.array(xi) for xi in chan_array])
  return result

In [8]:
#generate y for SVM
def get_y_val(subdat):
  y = subdat['stim_id']
  return y

In [9]:
#recode all the label of the stimulus id
def label_recode(dataf):
  new_stim_id_array = np.zeros(60)
  for stim_id , i in zip(dataf['stim_id'],range(0,60)) :
    if stim_id == 11 :
      new_stim_id_array[i] = 0
    else :
      new_stim_id_array[i] = 1
    return new_stim_id_array

In [11]:
#get data from subject 0, experiment 0
dat = get_sub_dat(alldat, 0, 0)
datf = dat_frame_t(dat, channel_n = 46)
dftime = dftime_win(dat)
# get X and y input for the SVM
X1 = slice_chan_dat(datf, dftime, 44)
X2 = slice_chan_dat(datf, dftime, 30)
y = get_y_val(dat)
# y = label_recode(dat)
# print(dat)
# X_stack = np.stack((X1, X2), axis=0)

[[-2.375  -2.426  -2.34   ...  0.4834  0.611   0.6304]
 [-0.534  -0.5444 -0.4182 ...  0.7676  0.897   0.9385]
 [-2.016  -2.078  -2.002  ...  1.211   1.275   1.172 ]
 ...
 [ 0.474   0.4614  0.3918 ...  0.55    0.5884  0.6714]
 [-1.405  -1.316  -1.309  ... -1.19   -1.177  -1.132 ]
 [ 1.412   1.322   1.125  ...  1.333   1.345   1.301 ]]
[[-1.464  -1.461  -1.378  ...  1.719   1.773   1.766 ]
 [-2.803  -2.854  -2.775  ...  1.076   1.186   1.183 ]
 [ 1.185   1.177   1.21   ...  0.4773  0.507   0.3508]
 ...
 [ 0.1771  0.2197  0.2362 ... -0.647  -0.6543 -0.5967]
 [ 0.3496  0.376   0.39   ... -0.625  -0.6323 -0.6294]
 [ 1.159   1.169   1.142  ...  0.863   0.8945  0.913 ]]


In [ ]:
clf = svm.SVC(gamma=0.001 , C=100)
clf.fit(X1[0:50,:],y[:50])

SVC(C=100, gamma=0.001)

In [ ]:
#!!!DONT RUN!!! find feature selection library, still doing
def channel_comp(dat, channel_n):
  eligible_chan_id = []
  datf = dat_frame_t(dat, channel_n = 46)
  dftime = dftime_win(dat)
  for i in range(channel_n):
    for j in range(channel_n):
      X1 = slice_chan_dat(datf, dftime, i)
      X2 = slice_chan_dat(datf, dftime, j)
      y = get_y_val(dat)
  return eligible_chan_id